# Demonstration of LangChain RAG, Input/Output Parsing, and Structured Responses



This section ensures all required libraries are installed before running the notebook.

- `langchain` → Used for integrating OpenAI models and FAISS.
- `openai` → Allows us to use OpenAI’s GPT models for generating responses.
- `faiss-cpu` → Used for storing and retrieving vector embeddings.
- `pymupdf` → Helps extract text from PDF documents.
- `sentence-transformers` → Converts text into vector embeddings.

In [16]:
# Install Dependencies
!pip install langchain openai faiss-cpu pymupdf gdown
!pip install -U langchain-community
!pip install tiktoken
!pip install --upgrade openai langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.4 MB/s eta 0:00:00


In [18]:
# Import Required Libraries
import fitz  # PyMuPDF for PDF processing
import gdown  # For downloading resume
import os
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Building an AI-Powered Resume Q&A System with LangChain

###  **This Notebook Demonstrates**
This notebook showcases how to build an **AI-powered Q&A system** that allows users to ask questions about a resume document using **LangChain** and **Retrieval-Augmented Generation (RAG)**.



* **Retrieval-Augmented Generation (RAG)** – Using FAISS to retrieve relevant resume content before querying OpenAI’s LLM.  
* **Input Parsing & Processing** – Cleaning user queries and formatting them for effective retrieval.  
* **Output Formatting** – Ensuring structured, readable, and relevant answers.  
* **Interactive Q&A System** – Allowing users to ask dynamic questions and receive context-aware answers.


## **Load and Process Resume Data**

- Extract text from a resume PDF.
- Convert the text into a structured format for retrieval.

In [19]:
# Load and Process a resume
resume_url = "https://drive.google.com/uc?id=1WxN09mL1KuBQc0VIYm1nVF38Sd0gTMSq"
resume_path = "/content/resume.pdf"
gdown.download(resume_url, resume_path, quiet=False)

# Open PDF
try:
    doc = fitz.open(resume_path)
    print("✅ PDF opened successfully!")
except Exception as e:
    print(f"❌ PDF might be corrupted! Error: {e}")

# Extract Text from Resume
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

resume_text = extract_text_from_pdf(resume_path)

Downloading...
From: https://drive.google.com/uc?id=1WxN09mL1KuBQc0VIYm1nVF38Sd0gTMSq
To: /content/resume.pdf
100%|██████████| 92.8k/92.8k [00:00<00:00, 66.8MB/s]

✅ PDF opened successfully!


## Converting Text to Embeddings & Storing in FAISS

To make search efficient, we **convert text into vector embeddings** using `SentenceTransformers` and **store them in FAISS**.


### Steps in this Section
* Load a **pretrained embedding model** (`all-MiniLM-L6-v2`).  
* Convert **resume text into vector embeddings**.  
* Store these embeddings in a **FAISS vector index**.  
* Save the FAISS index for **future retrieval**.


In [22]:
# Create Embeddings and FAISS Vector Store
from sentence_transformers import SentenceTransformer

# Load SentenceTransformer model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Define a wrapper to make it compatible with FAISS
class CustomEmbeddings:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        """Encodes a list of texts correctly."""
        if not isinstance(texts, list):
            raise ValueError("Input to embed_documents() must be a list of strings.")
        return self.model.encode(texts)

    def embed_query(self, text):
        """Encodes a single query correctly."""
        if not isinstance(text, str):
            raise ValueError("Input to embed_query() must be a string.")
        return self.model.encode([text])[0]  # Ensure a single embedding is returned

    def __call__(self, text):
        """Makes the class instance callable."""
        return self.embed_query(text)

# Instantiate Embedding Wrapper
embedding_function = CustomEmbeddings(embedding_model)

In [23]:
from langchain.vectorstores import FAISS
import numpy as np

# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Wrap embedding model
embedding_function = CustomEmbeddings(embedding_model)

# Convert resume text into an embedding
resume_embedding = embedding_function.embed_documents([resume_text])

# Convert embeddings into a NumPy array
resume_embedding = np.array(resume_embedding)

# Create FAISS Vector Store with Correct Function
vector_store = FAISS.from_texts([resume_text], embedding_function)

# Save FAISS Index
vector_store.save_local("faiss_index")
print("✅ FAISS Index Created & Saved!")

✅ FAISS Index Created & Saved!


## Load OpenAI Model & Set Up RAG Pipeline

In this section, we integrate OpenAI’s **GPT model** to generate **fact-based answers** using **retrieved context**.

### Why Use RAG?
- Instead of asking OpenAI a question blindly, **we first retrieve relevant information** from FAISS.
- This makes the responses **more accurate and factually grounded**.

### Steps in this Section
* Load **gpt-4o-mini**  
* Retrieve **relevant resume content** using FAISS.  
* Feed the **retrieved content** to GPT before generating a response.  
* Ensure **accurate & context-aware responses**.

In [29]:
#Load OpenAI Model and Set Up RAG Pipeline
import os
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from getpass import getpass

# Load FAISS Index with safe deserialization
vector_store = FAISS.load_local("faiss_index", embedding_function, allow_dangerous_deserialization=True)

# Prompt the user for their OpenAI API key
api_key = getpass("Enter your OpenAI API key: ")

# Set the environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Use an available model instead of 'gpt-4'
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Ensure FAISS is correctly loaded
vector_store = FAISS.load_local("faiss_index", embedding_function, allow_dangerous_deserialization=True)

# Corrected RetrievalQA initialization
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vector_store.as_retriever())

print("✅ RetrievalQA Chain Ready!")

Enter your OpenAI API key: ··········


✅ RetrievalQA Chain Ready!


### Formatting AI Responses for Better Readability

AI responses need **clear structure and formatting**. Here, we use `StructuredOutputParser` to ensure responses follow a structured format.

In [30]:
# Define Output Formatting with Response Schema
response_schemas = [
    ResponseSchema(name="answer", description="The detailed answer based on resume"),
    ResponseSchema(name="source", description="The section of the resume where the information was found"),
]

# Initialize Structured Parser
parser = StructuredOutputParser.from_response_schemas(response_schemas)
print("✅ Output Formatting Set Up!")

✅ Output Formatting Set Up!


In [31]:
# Define Query Function

def ask_question(query):
    if not isinstance(query, str):
        raise ValueError("Query must be a string.")

    cleaned_query = query.strip()  # Remove extra spaces

    # Corrected: Call embed_query explicitly
    query_embedding = embedding_function.embed_query(cleaned_query)

    # Retrieve relevant documents using FAISS
    docs = vector_store.similarity_search_by_vector(query_embedding, k=3)

    # Query the LLM
    response = qa_chain.invoke(cleaned_query)

    return {
        "answer": response["result"],  # Ensure correct key
        "source": [doc.page_content for doc in docs]  # Extract text from retrieved documents
    }

In [32]:
query = "Where do I currently study?"
result = ask_question(query)

# Display Answer
print("\nQuestion:", query)
print("Answer:", result["answer"])
print("Source:", result["source"])


Question: Where do I currently study?
Answer: You currently study at Northeastern University in Boston, MA.
Source: ['Srishti C Rai\nBoston, MA \x83 (413) 419-6505 # srishtiraic@gmail.com ï linkedin.com/in/srishti-c-rai § github.com/srishtirai\nEDUCATION\nNortheastern University\nExpected Jul 2025\nMaster of Science, Software Engineering Systems (3.8 GPA)\nBoston, MA\nRNS Institute of Technology\nAug 2020\nBachelor of Engineering, Computer Science & Engineering\nBangalore, India\nTECHNICAL SKILLS\n• Programming Languages: Java, JavaScript, TypeScript, Python, Dart, C#\n• Frontend Development: HTML, CSS, React, Next.js, Sass, Bootstrap, Redux, AJAX, Webpack, Flutter, Figma\n• Backend Development: JSP, Flask, REST API, Spring Boot, JSON, Nest.js, ASP.NET\nTesting: Selenium, Jest\n• Database: SQL, MySQL, PostgreSQL, MSSQL Cloud: AWS (S3, IAM, EC2), Azure DevOps: Docker, Octopus\nEXPERIENCE\nSoftware Development Intern\nMay 2024 – Dec 2024\nSentinel Group\nWakefield, MA\n• Developed a Nes

### Interactive Q&A with User Input

This section allows **users to dynamically ask questions** and get fact-based answers.

* The system **continuously waits for user input**.  
* The input question is **converted into an embedding**.  
* FAISS **retrieves relevant documents** from the resume.  
* The retrieved content is **fed into OpenAI** for an accurate response.  
* The answer and **source document** are printed.

In [33]:
def interactive_qa():
    while True:
        query = input("\nAsk a question (or type 'exit' to quit): ").strip()

        if query.lower() == "exit":
            print("Exiting Interactive Q&A. Goodbye!")
            break

        # Get the answer using the RAG pipeline
        result = ask_question(query)

        # Print the response
        print("\nAnswer:", result["answer"])
        print("Source:", result["source"])

# Run the interactive Q&A
interactive_qa()



Ask a question (or type 'exit' to quit): Summarise my work experince and skills

Answer: You have a strong background in software development with experience in both full-stack and backend roles. 

**Work Experience:**
1. **Software Development Intern at Sentinel Group (May 2024 – Dec 2024)**: Developed a NestJS solution for automating folder creation and permissions, engineered an audit feature for compliance verification, and created an admin interface using Next.js.
   
2. **Research Engineer (Full Stack Developer) at LG Soft India (Jul 2021 – Aug 2023)**: Built a React application for OTA updates, collaborated with UI/UX teams in South Korea, designed secure APIs in Spring Boot, optimized AWS S3 uploads, and mentored interns.

3. **Software Engineer at LG Soft India (Jul 2020 – Jun 2021)**: Developed applications for the webOS ecosystem using EnactJS and created a Flutter proof of concept.

**Technical Skills:**
- **Programming Languages**: Java, JavaScript, TypeScript, Python, Da